[View in Colaboratory](https://colab.research.google.com/github/pgurazada/Exploring-Deep-Learning/blob/master/2018-06-12_fine-tuning-convnets.ipynb)

### Connect Google Drive and provide access to data

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [0]:
import getpass

In [0]:
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive/data

### Fine tune the pre-trained VGG model

1.  Add your custom network on top of an already-trained base network.
2. Freeze the base network.
3. Train the part you added.
4. Unfreeze some layers in the base network.
5. Jointly train both these layers and the part you added.

It is a good idea to fine tune only the top two - three layers of a pretrained neural net

In [0]:
from keras.applications import VGG16

In [0]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [0]:
conv_base.summary()

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_dir = 'drive/data/train'
validation_dir = 'drive/data/validation'

In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

In [0]:
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

In [0]:
from keras.models import Sequential

from keras.layers import Dense, Flatten, Activation

from keras.losses import binary_crossentropy

from keras.optimizers import RMSprop

from keras.metrics import binary_accuracy

from keras.callbacks import EarlyStopping

from keras import backend as K

### Train the last block of convolutions

In [0]:
conv_base.trainable = True

set_trainable = False

In [0]:
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True # this ensures that all layers starting from conv1 in block 5 are now set to trainable
  
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [0]:
model = Sequential()

model.add(conv_base)

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
model.summary()

In [0]:
model.compile(loss=binary_crossentropy,
              optimizer=RMSprop(lr=1e-5),
              metrics=[binary_accuracy])

In [0]:
model_output = model.fit_generator(train_generator,
                                   steps_per_epoch=100,
                                   epochs=100,
                                   validation_data=validation_generator,
                                   validation_steps=50,
                                   callbacks=[EarlyStopping(patience=4)])

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set_style('ticks')
sns.set_context('talk', font_scale=1.4)

In [0]:
plt.plot(model_output.history['val_binary_accuracy'], label='Validation')
plt.plot(model_output.history['binary_accuracy'], '--', label='Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

In [0]:
plt.plot(model_output.history['val_loss'], label='Validation')
plt.plot(model_output.history['loss'], '--', label='Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [0]:
model.save('drive/Colab Notebooks/cats_and_dogs_small_4.h5')

### Fine tuning the top two layers

In [0]:
K.clear_session()

In [0]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [0]:
conv_base.trainable = True

set_trainable = False

In [0]:
for layer in conv_base.layers:
  if layer.name == 'block4_conv1':
    set_trainable = True # this ensures that all layers starting from conv1 in block 4 are now set to trainable
  
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [0]:
model = Sequential()

model.add(conv_base)

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
model.summary()

In [0]:
model.compile(loss=binary_crossentropy,
              optimizer=RMSprop(lr=1e-5),
              metrics=[binary_accuracy])

In [0]:
model_output = model.fit_generator(train_generator,
                                   steps_per_epoch=100,
                                   epochs=100,
                                   validation_data=validation_generator,
                                   validation_steps=50,
                                   callbacks=[EarlyStopping(patience=4)])

In [0]:
plt.plot(model_output.history['val_binary_accuracy'], label='Validation')
plt.plot(model_output.history['binary_accuracy'], '--', label='Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

In [0]:
plt.plot(model_output.history['val_loss'], label='Validation')
plt.plot(model_output.history['loss'], '--', label='Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

The results are noisy but astounding given that we are using a small amount of data